In [1]:
import sys 
#sys.path.append("codes/modules") # add custom Vibe 's modules
sys.path.append('../..') # add standard 's modules
sys.path.append('../modules')

import pyfesom as pf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import numpy as np
from netCDF4 import Dataset
import os
import time

sys.path.append('/home/hbkoziel/pyfesom/pyfesom/python-gsw/')
import gsw

No joblib
no cmocean


In [2]:
# ==============================================================================
# Running this file loads tracers from old FESOM-REcoM2 output file (oce.mean.nc)
# and saves each tracer in an individual file
# 
#  Input:
#  - mesh_id: Name of mesh, will be added to the netcdf name
#  - meshpath: Speciefies where the target mesh is stored
#  - save_netcdf: If true, netcdf will be created
#  - delete_old_netcdf: If a netcdf file with the same name exists, a new cannot 
#    be made. If set to true, an old netcdf with the same name will be deleted
#
#  Output:
#  - netcdf file for each tracer in the old file
#  
#  During running, keep an eye on the output in the terminal, to see if it 
#  makes sense. 
#
# ==============================================================================

In [3]:
def Turner_Rsubrho(SA, CT, p):
    SA, CT, p = np.broadcast_arrays(SA, CT, p, subok=True)
    p_mid = 0.5 * (p[0:-1, ...] + p[1:, ...])
    SA_mid = 0.5 * (SA[0:-1, ...] + SA[1:, ...])
    CT_mid = 0.5 * (CT[0:-1, ...] + CT[1:, ...])

    dSA = SA[0:-1, ...] - SA[1:, ...]
    dCT = CT[0:-1, ...] - CT[1:, ...]
    [rho, alpha, beta] = gsw.rho_alpha_beta(SA_mid, CT_mid, p_mid)
    #Tu = np.arctan2((alpha * dCT + beta * dSA), (alpha * dCT - beta * dSA))
    #Tu = Tu * (180 / np.pi)

    Rsubrho = np.zeros_like(dSA) + np.NaN
    Inz = dSA != 0
    Rsubrho[Inz] = (alpha[Inz] * dCT[Inz]) / (beta[Inz] * dSA[Inz])
    return Rsubrho, rho

In [4]:
# Loading mesh for run

mesh_id    = 'meshArc4.5'
meshpath   = '/scratch/usr/hbkoziel/mesh/'+mesh_id+'/'            # Defining path where mesh is stored
# mesh = pf.load_mesh(meshpath, usepickle=False, get3d=True)                                    # Loading mesh, stores it in mesh.****  
mesh = pf.load_mesh(meshpath, usepickle=True, get3d=True)
#mesh.zlevs = -mesh.zlevs                                            # Depth is made negative

tracername = 'SAPE'
first_year = 2013
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
runid	= 'Arc12'

/scratch/usr/hbkoziel/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /scratch/usr/hbkoziel/mesh/meshArc4.5/pickle_mesh


In [5]:
mesh


FESOM mesh:
path                  = /home/hbkoziel/pyfesom/data/mesh/meshArc4.5
alpha, beta, gamma    = 50, 15, -90
number of 2d nodes    = 753179
number of 2d elements = 1480268
number of 3d nodes    = 16950142

        

In [6]:
pressure = gsw.p_from_z(-mesh.z3, mesh.y3, geo_strf_dyn_height=0)

In [7]:
# ==============================================================================
# Settings for netcdf file

save_netcdf       = True                                            # Saves the interpolated field in netcdf file
delete_old_netcdf = True                                            # If a netcdf file with the same name exists it will be deleted
input_directory  = '/scratch/usr/hbkoziel/'+runid+'/netcdf_monthly/'       # Where the netcdf is saved
output_directory  = '/scratch/usr/hbkoziel/'+runid+'/netcdf_monthly/'
plot_netcdf       = True                                           # Reads DIN from the created netcdf file, else it plots the interpolated field (should be the same)

In [8]:
for ind in range(0,len(years)):
    netcdf_name       = tracername+'.'+str(years[ind])+'.monthly.nc'
    print years[ind]
    # ==============================================================================
    # Loading data

    ncfile1	= input_directory+'temp.'+str(years[ind])+'.monthly.nc'
    f1	= Dataset(ncfile1,'r')
    t	= f1.variables['temp'][8,:]
    f1.close()
    
    ncfile2	= input_directory+'salt.'+str(years[ind])+'.monthly.nc'
    f2	= Dataset(ncfile2,'r')
    s	= f2.variables['salt'][8,:]
    f2.close()
    
    ncfile3	= input_directory+'mixlay.'+str(years[ind])+'.monthly.nc'
    f3	= Dataset(ncfile3,'r')
    mld	= f3.variables['mixlay'][8,:]
    f3.close()
    
    HLD = np.zeros(len(mesh.x2))
    HLT = np.zeros(len(mesh.x2))
    APE = np.zeros(len(mesh.x2))
    for i in range(0,len(mesh.x2)):
        if ((mesh.y2[i] >= 70) & (mesh.topo[i]>300)):
            #print 'derive HL'
            #ind = (np.abs(mesh.topo[i]-mesh.zlevs)).argmin(axis=0)
            ind_depth = np.array(mesh.n32[i,0:20]) # depth max 580m
            ind_depth = np.reshape(ind_depth, ind_depth.size)
                
            z = pressure[ind_depth]
            
            tt = t[ind_depth]
            tt = np.reshape(tt, tt.size)
            ss = s[ind_depth]
            ss = np.reshape(ss, ss.size)
            [Ro1,ro] = Turner_Rsubrho(ss,tt,z)
            hl_ind = (np.abs(0.1-Ro1)).argmin(axis=0)
            HLD[i] = mesh.z3[ind_depth[hl_ind]]
            if HLD[i] > mld[i]:
                HLT[i] = HLD[i] - mld[i]
            else:
                HLT[i] = 0
            APE[i] = np.trapz(HLD[i]*9.81*(ro[0:hl_ind+1]-ro[hl_ind]),-mesh.z3[ind_depth[:hl_ind+1]])
        
        #tracer = [HL1,HL2,HL3,HL4,HL5,HL6,HL7,HL8,HL9,HL10,HL11,HL12]
            
    tracershape = np.shape(HLD)
    
    # ==============================================================================
    # Testing if a netcdf file with the same name exists, if yes, it must be removed
    # to save a new one.

    if os.path.isfile(output_directory+netcdf_name) and delete_old_netcdf:
      os.remove(output_directory+netcdf_name)
      print "The netcdf file "+netcdf_name+" has been deleted to make room for your file of the same name."
    elif os.path.isfile(netcdf_name):
      statement = "The netcdf file "+netcdf_name+" already exists! It must be removed for a new one to be created. This can be done by changing your settings."
      sys.exit(statement)

    if not os.path.isdir(output_directory):
      os.makedirs(output_directory)
      print 'Directory '+output_directory+' has been created'

    # ==============================================================================
    # Creating netcdf file
    if save_netcdf:  
      import time
      w_nc_fid = Dataset(output_directory+netcdf_name, 'w', format='NETCDF4_CLASSIC')      # Create and open new netcdf file to write to
      w_nc_fid.description = 'Mean Summer (Sept) Halocline indicators'
      w_nc_fid.history     = 'Created ' + time.ctime(time.time())

      nod2d    = w_nc_fid.createDimension('nod2d', mesh.n2d)               # Create dimension: number of 3d nodes

      w_nc_var = w_nc_fid.createVariable('HLD', 'f4',('nod2d'))           # 'DIN' is name of saved variable                                                                    # 'f8' sets presicion to 64-bit floating point
      w_nc_var.setncatts({'long_name': u'Mean Summer Halocline Depth',\
                          'units': u'm'})
      w_nc_fid.variables['HLD'][:] = HLD  
    
      w_nc_var = w_nc_fid.createVariable('HLT', 'f4',('nod2d'))           # 'DIN' is name of saved variable                                                                    # 'f8' sets presicion to 64-bit floating point
      w_nc_var.setncatts({'long_name': u'Mean Summer Halocline Thickness',\
                          'units': u'm'})
      w_nc_fid.variables['HLT'][:] = HLT
        
      w_nc_var = w_nc_fid.createVariable('APE', 'f4',('nod2d'))           # 'DIN' is name of saved variable                                                                    # 'f8' sets presicion to 64-bit floating point
      w_nc_var.setncatts({'long_name': u'Mean Summer Available Potential Energy',\
                          'units': u'J/m2'})
      w_nc_fid.variables['APE'][:] = APE 
    
    
      w_nc_fid.close()                                                     # close the new file                

      cwd = os.getcwd()
      print "New netcdf file (",netcdf_name,") has been created."
      print "Location: "+output_directory
    else:
      print 'You have specified not to save your field in netcdf file'

2013


/sw/tools/anaconda2/2019.10/skl/lib/python2.7/site-packages/ipykernel_launcher.py:45: UserWarning: Warning: converting a masked element to nan.


The netcdf file SAPE.2013.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( SAPE.2013.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
2014
The netcdf file SAPE.2014.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( SAPE.2014.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/
2015
The netcdf file SAPE.2015.monthly.nc has been deleted to make room for your file of the same name.
New netcdf file ( SAPE.2015.monthly.nc ) has been created.
Location: /scratch/usr/hbkoziel/Arc12/netcdf/


In [9]:
# data2, elem_no_nan = pf.get_data(HLD,mesh,0)
# data2=np.copy(data2)
# print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
# print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
# print 'Mean of surface: ',np.mean(data2)
# print 'Max and min: ',np.max(data2),np.min(data2)


# contours = np.arange(0, 600, 10)

# fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
# #fig.suptitle('Density (kg m$^{-3}$)')
# #fig.subplots_adjust(wspace=0.02,hspace=0.02)

# m = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
# x, y = m(mesh.x2, mesh.y2)
# #map.drawcoastlines()
# plabels=[False,False,False,False]
# mlabels=[False,False,False,False]    
# m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
# m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
# m.drawmapboundary(fill_color='0.9')
# m.fillcontinents(color='.5',lake_color='.7')

# #eps=(contours.max()-contours.min())/100.
# #data2[data2<=contours.min()]=contours.min()+eps
# #data2[data2>=contours.max()]=contours.max()-eps

# im=plt.tricontourf(x, y, elem_no_nan, data2, levels=contours, cmap=cm.viridis, extend='max')

# label = 'HLD (m)'
# #plt.title(year)
# cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
# cbar.set_label(label)

# #plt.savefig(saving_directory+'ChlNano.png', dpi = 200, bbox_inches='tight')
# plt.show()

In [10]:
# data2, elem_no_nan = pf.get_data(HLT,mesh,0)
# data2=np.copy(data2)
# print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
# print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
# print 'Mean of surface: ',np.mean(data2)
# print 'Max and min: ',np.max(data2),np.min(data2)

# contours = np.arange(0, 600, 10)

# fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
# #fig.suptitle('Density (kg m$^{-3}$)')
# #fig.subplots_adjust(wspace=0.02,hspace=0.02)

# m = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
# x, y = m(mesh.x2, mesh.y2)
# #map.drawcoastlines()
# plabels=[False,False,False,False]
# mlabels=[False,False,False,False]    
# m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
# m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
# m.drawmapboundary(fill_color='0.9')
# m.fillcontinents(color='.5',lake_color='.7')

# eps=(contours.max()-contours.min())/100.
# data2[data2<=contours.min()]=contours.min()+eps
# data2[data2>=contours.max()]=contours.max()-eps

# im=plt.tricontourf(x, y, elem_no_nan, data2, levels=contours, cmap=cm.viridis, extend='max')

# label = 'HLT (m)'
# #plt.title(year)
# cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
# cbar.set_label(label)

# #plt.savefig(saving_directory+'ChlNano.png', dpi = 200, bbox_inches='tight')
# plt.show()

In [11]:
# data2, elem_no_nan = pf.get_data(APE,mesh,0)
# data2=np.copy(data2)
# data2 = data2 /10**5
# print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
# print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
# print 'Mean of surface: ',np.nanmean(data2)
# print 'Max and min: ',np.nanmax(data2),np.nanmin(data2)

# contours = np.arange(0, 30, 1)

# fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
# #fig.suptitle('Density (kg m$^{-3}$)')
# #fig.subplots_adjust(wspace=0.02,hspace=0.02)

# m = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
# x, y = m(mesh.x2, mesh.y2)
# #map.drawcoastlines()
# plabels=[False,False,False,False]
# mlabels=[False,False,False,False]    
# m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
# m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
# m.drawmapboundary(fill_color='0.9')
# m.fillcontinents(color='.5',lake_color='.7')

# eps=(contours.max()-contours.min())/100.
# data2[data2<=contours.min()]=contours.min()+eps
# data2[data2>=contours.max()]=contours.max()-eps

# im=plt.tricontourf(x, y, elem_no_nan, data2, levels=contours, cmap=cm.viridis, extend='max')

# label = 'APE (J m-2)'
# #plt.title(year)
# cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
# cbar.set_label(label)

# #plt.savefig(saving_directory+'ChlNano.png', dpi = 200, bbox_inches='tight')
# plt.show()

In [12]:
# data2, elem_no_nan = pf.get_data(mld,mesh,0)
# data2=np.copy(data2)
# print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
# print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
# print 'Mean of surface: ',np.mean(data2)
# print 'Max and min: ',np.max(data2),np.min(data2)

# contours = np.arange(0, 600, 10)

# fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
# #fig.suptitle('Density (kg m$^{-3}$)')
# #fig.subplots_adjust(wspace=0.02,hspace=0.02)

# m = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
# x, y = m(mesh.x2, mesh.y2)
# #map.drawcoastlines()
# plabels=[False,False,False,False]
# mlabels=[False,False,False,False]    
# m.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
# m.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
# m.drawmapboundary(fill_color='0.9')
# m.fillcontinents(color='.5',lake_color='.7')

# eps=(contours.max()-contours.min())/100.
# data2[data2<=contours.min()]=contours.min()+eps
# data2[data2>=contours.max()]=contours.max()-eps

# im=plt.tricontourf(x, y, elem_no_nan, data2, levels=contours, cmap=cm.viridis, extend='max')

# label = 'MLD (m)'
# #plt.title(year)
# cbar=m.colorbar(im,"bottom", size="5%", pad="2%")
# cbar.set_label(label)

# #plt.savefig(saving_directory+'ChlNano.png', dpi = 200, bbox_inches='tight')
# plt.show()